In [22]:
from torchvision import transforms
import torch.nn.functional as F
import torch

from tqdm import tqdm_notebook as tqdm

from hw_grapheme.model_archs.se_resnext import se_resnext50
from hw_grapheme.io.load_model_weight import load_model_weight
from hw_grapheme.io.load_data import load_processed_data
from hw_grapheme.train_utils.train_test_split import stratified_split_kfold
from hw_grapheme.train_utils.create_dataloader import create_dataloaders_train


In [32]:
import numpy as np
import sklearn
from sklearn.metrics import classification_report, precision_recall_fscore_support
import pandas as pd

def flatten(ls):
    array = np.array(ls)
    return np.hstack(array.tolist())

def show_most_wrong(df,n=10):
    return df.sort_values('recall').head(10)

In [25]:
valid_fold = 0
batch_size = 256
num_workers = 6
pin_memory = True


In [13]:
# load processed data 
pickle_paths = [
    "../data/processed/size_128/train_data_0.pickle",
    "../data/processed/size_128/train_data_1.pickle",
    "../data/processed/size_128/train_data_2.pickle",
    "../data/processed/size_128/train_data_3.pickle",
]

image_data, name_data, label_data = load_processed_data(pickle_paths, image_size=128)

# split train valid set
n_splits = 5
random_seed = 2020

train_idx_list, test_idx_list = stratified_split_kfold(
    image_data, label_data, n_splits, random_seed
)

train_idx = train_idx_list[valid_fold]
valid_idx = test_idx_list[valid_fold]

# create data_transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
    ]),
}

data_loaders = create_dataloaders_train(
    image_data, name_data, label_data, train_idx, valid_idx, 
    data_transforms, batch_size, num_workers, pin_memory
)

Load data done, shape: (200840, 128, 128), (200840,), (200840, 3)
StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
Creating train dataloader...
Creating valid dataloader...


In [15]:
model = se_resnext50()
load_model_weight(model, "../models/se_resnext_sgd_decay/fold_0/high_recall_model.pth")
model.cuda();

In [17]:
ls = []
y1_ls = []
y2_ls = []
y3_ls = []
y1_pred_ls = []
y2_pred_ls = []
y3_pred_ls = []

In [23]:
for i in tqdm(data_loaders['val']):
    x,y1,y2,y3 = i 
    with torch.no_grad():
        y_pred = model(x.cuda())
        y1_pred, y2_pred ,y3_pred = y_pred

        y1_pred, y2_pred, y3_pred = map(lambda x:F.softmax(torch.tensor(x)).argmax(1), (y1_pred, y2_pred, y3_pred))
        y1_pred = y1_pred.detach().cpu().numpy()
        y2_pred = y2_pred.detach().cpu().numpy()
        y3_pred = y3_pred.detach().cpu().numpy()
        y1= y1.detach().cpu().numpy()
        y2=y2.detach().cpu().numpy()
        y3=y3.detach().cpu().numpy()
        
        ls.append(y_pred)
        y1_ls.append(y1)
        y2_ls.append(y2)
        y3_ls.append(y3)
        y1_pred_ls.append(y1_pred)
        y2_pred_ls.append(y2_pred)
        y3_pred_ls.append(y3_pred)

/home/lam/anaconda3/envs/hugo/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/home/lam/anaconda3/envs/hugo/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/lam/anaconda3/envs/hugo/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [27]:
y1_ls = flatten(y1_ls)
y2_ls = flatten(y2_ls)
y3_ls = flatten(y3_ls)
y1_pred_ls = flatten(y1_pred_ls)
y2_pred_ls = flatten(y2_pred_ls)
y3_pred_ls = flatten(y3_pred_ls)

In [30]:
y_trues , y_preds = [y1_ls, y2_ls, y3_ls], [y1_pred_ls, y2_pred_ls, y3_pred_ls]

In [31]:
for y_true, y_pred in zip(y_trues, y_preds):
    print(sklearn.metrics.recall_score(y_true, y_pred, average='macro', zero_division='warn'))
    

0.9542337685380651
0.9864964429113378
0.9805598165609315


In [33]:
for y_true, y_pred in zip(y_trues, y_preds):
    metrics = precision_recall_fscore_support(y_true, y_pred,average=None)
    precision, recall, fscore, support = metrics
    data = np.vstack([recall, support, precision, fscore]); data.shape    
    metrics_summary_df = pd.DataFrame(data=data.T, columns=['recall', 'support','precision','fscore'])
    metrics_summary_df = metrics_summary_df.sort_values('recall')
    print(show_most_wrong(metrics_summary_df))
    print("================= I am separation line ============")

       recall  support  precision    fscore
60   0.607143     84.0   0.809524  0.693878
61   0.704348    115.0   0.778846  0.739726
62   0.782857    175.0   0.801170  0.791908
84   0.791209     91.0   0.699029  0.742268
162  0.795699     93.0   0.880952  0.836158
145  0.828125     64.0   0.981481  0.898305
68   0.856000    125.0   0.914530  0.884298
54   0.864407    118.0   0.918919  0.890830
63   0.866667     30.0   0.896552  0.881356
116  0.873563     87.0   0.904762  0.888889
================= I am separation line ============
      recall  support  precision    fscore
6   0.970012    867.0   0.980186  0.975072
5   0.973585   1060.0   0.973585  0.973585
8   0.978784    707.0   0.991404  0.985053
10  0.985975    713.0   0.990141  0.988053
4   0.987003   3770.0   0.988839  0.987920
0   0.989265   8291.0   0.986766  0.988014
9   0.989704   3205.0   0.987547  0.988624
1   0.990785   7379.0   0.991188  0.990986
3   0.993193   3232.0   0.996276  0.994732
7   0.993734   5745.0   0.992524  